In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data16 = pd.read_csv('../input/train_2016_v2.csv',parse_dates=['transactiondate'],low_memory = False)
data16.shape


In [ ]:
prop16 = pd.read_csv('../input/properties_2016.csv',low_memory=False)
prop16.shape

In [ ]:
data17 = pd.read_csv('../input/train_2017.csv',parse_dates=['transactiondate'],low_memory = False)
data17.shape

In [ ]:
prop17 = pd.read_csv('../input/properties_2017.csv',low_memory=False)
prop17.shape

In [ ]:
import datetime as dt

data16["transaction_year"] = data16["transactiondate"].dt.year
data16["transaction_month"] =data16["transactiondate"].dt.month
data16["transaction_day"] = data16["transactiondate"].dt.day
data16.drop(["transactiondate"], inplace=True, axis=1)

data17["transaction_year"] = data17["transactiondate"].dt.year
data17["transaction_month"] =data17["transactiondate"].dt.month
data17["transaction_day"] = data17["transactiondate"].dt.day
data17.drop(["transactiondate"], inplace=True, axis=1)
print(data16)

In [ ]:
#Fill missing in prop16
prop16.fillna(-111,inplace = True)
print(prop16)
#Fill missing in prop17
prop17.fillna(-111,inplace = True)
print(prop17)


In [ ]:

train2016 = pd.merge(data16, prop16, how = 'left', on = 'parcelid')
train2017 = pd.merge(data17, prop17, how = 'left', on = 'parcelid')
template = pd.read_csv('../input/sample_submission.csv', low_memory = False)
#train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan

train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(template[['ParcelId']], prop16.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')


In [ ]:
train_df.shape

In [ ]:
for dtype in train_df.columns:
    if dtype == np.float64:
        train_df[c] = train_df[c].astype(np.float32)
train_df.info()

In [ ]:
from sklearn import preprocessing
for feature in train_df.columns:
    if train_df[feature].dtype=='object':
        label = preprocessing.LabelEncoder()
        label.fit(list(train_df[feature].values)) 
        train_df[feature] = label.transform(list(train_df[feature].values))
train_df.info()     

In [ ]:
dftrain = train_df
xtrain = dftrain.drop(['parcelid', 'logerror', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
ytrain = dftrain['logerror'].values
xtrain,ytrain

In [ ]:
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df["transaction_year"] = test_df["transactiondate"].dt.year
test_df["transaction_month"] =test_df["transactiondate"].dt.month
test_df["transaction_day"] = test_df["transactiondate"].dt.day
test_df.drop(["transactiondate"], inplace=True, axis=1)

test_df.info()

In [ ]:
from sklearn import preprocessing
for feature in test_df.columns:
    if test_df[feature].dtype=='object':
        label = preprocessing.LabelEncoder()
        label.fit(list(test_df[feature].values)) 
        test_df[feature] = label.transform(list(test_df[feature].values))
train_df.info()    

temp=[]
for col in xtrain.columns:
    temp.append(col)
temp

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
regressor=AdaBoostRegressor()
#regressor.fit(xtrain,ytrain)

#from catboost import CatBoostClassifier
#regressor=CatBoostClassifier()

regressor.fit(xtrain,ytrain)
xtrain.shape

In [ ]:
test_df.shape
xtest=test_df[temp]
y_pred=regressor.predict(xtest)



In [ ]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})
test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred

In [ ]:
submission.to_csv('Only_CatBoost.csv', float_format='%.6f',index=False)